In [1]:
#%%
import os
import re
import sys
import time
import json

import numpy as np
import pandas as pd
from pandas.core.indexes.base import Index

from tqdm import tqdm


#%%
# !pip install chromedriver_autoinstaller
from bs4 import BeautifulSoup
import chromedriver_autoinstaller

import selenium
from selenium import webdriver
from selenium.webdriver import ActionChains

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select, WebDriverWait

from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, ElementNotInteractableException

# from html_table_parser import parser_functions as parser

pd.set_option('mode.chained_assignment', None)

In [24]:
#%%
url = 'https://www.echemportal.org/echemportal/property-search'

option = webdriver.ChromeOptions()
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
option.add_argument('window-size=1920,1080')

driver_path = '/home/rt/chromedriver'
driver = webdriver.Chrome(executable_path="/home/rt/chromedriver", chrome_options=chrome_options)
driver.implicitly_wait(3)
driver.get(url)


#@id는 F12에서 id라고 뜨는 위치다. 그리고 /div는 그 뒤에 몇개의 div가 있냐이고, a[2]는 a _ngcontent-c13이 2개인데, 그중에 2번째꺼를 의미
deselect_path = '//*[@id="datasources-panel-1"]/div/div/div/a[2]'
driver.find_element_by_xpath(deselect_path).click()
time.sleep(0.5)

#위에꺼의 2번째에 보면 div가 아니라 echem-search-sources란 창이 하나 더 있다. 찾기 어렵네 ㄷㄷ 그래서 쭉 찾아보면 div들이 있는 것 중 2번째를 선택하고, 그 안에 체크박스가 있고, div
echa_path = '//*[@id="datasources-panel-1"]/div/echem-search-sources/div/div/div/div[2]/echem-checkbox/div'
driver.find_element_by_xpath(echa_path).click()
time.sleep(0.5)

# add Query Block 버튼
query_path = '//*[@id="property_query-builder-panel-1"]/div/echem-property-query-panel/div[2]/div[1]/echem-query-builder/div[2]/div/button'
driver.find_element_by_xpath(query_path).click()
time.sleep(0.5)

# Toxicological information 버튼
tox_button_path = '//*[@id="QU.SE.7-toxicological-information-header"]/div/div[1]/button'
driver.find_element_by_xpath(tox_button_path).click()
time.sleep(0.5)

# tox-to-reproduction-header
tox_to_reproduct_path = '//*[@id="QU.SE.7.6-genetic-toxicity-header"]/div/div[2]'
driver.find_element_by_xpath(tox_to_reproduct_path).click()
time.sleep(0.5)

# header 하위 3개의 창
tox_to_repro_path = '//*[@id="QU.SE.7.6-genetic-toxicity"]/div/div/div[1]/div[3]/button'
driver.find_element_by_xpath(tox_to_repro_path).click()
time.sleep(0.5)

/tmp/ipykernel_521/567339563.py:12: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path="/home/rt/chromedriver", chrome_options=chrome_options)


In [25]:
#옆에 새로운 Define이란 창이 열림
#맨 위의 Type of information
info_type_path = '//*[@id="property_query-builder-panel-1"]/div/echem-property-query-panel/div[2]/div[3]/echem-property-form/form/echem-property-phrase-field[1]/div/div/div/ng-select/div/div'
driver.find_element_by_xpath(info_type_path).click()
time.sleep(0.5)

# Type 중에서 experimental study를 선택
experiment_path = '/html/body/ng-dropdown-panel/div[2]/div[2]/div[3]'
driver.find_element_by_xpath(experiment_path).click()
time.sleep(0.5)

#Test guideline 선택
tg_path = '//*[@id="property_query-builder-panel-1"]/div/echem-property-query-panel/div[2]/div[3]/echem-property-form/form/echem-property-phrase-field[4]/div/div/div/ng-select/div/div'
driver.find_element_by_xpath(tg_path).click()
time.sleep(0.5)

#TG476경로, 이건 바꿔야함
tg476_path = '/html/body/ng-dropdown-panel/div[2]/div[2]/div[38]'
driver.find_element_by_xpath(tg476_path).click()
time.sleep(0.5)

#EPA8705300경로, 이것도 바꿔야함
epa8705300_path = '/html/body/ng-dropdown-panel/div[2]/div[2]/div[6]'
driver.find_element_by_xpath(epa8705300_path).click()
time.sleep(0.5)

#이건 저장이니 안건드려도 됨
save_path = '/html/body/echem-root/div/echem-substance-search-page/echem-substance-search-container/echem-substance-search/form/div/ngb-accordion[1]/div[2]/div[2]/div/echem-property-query-panel/div[2]/div[3]/echem-property-form/form/div/button[2]'
driver.find_element_by_xpath(save_path).click()
time.sleep(0.5)

#이것도 검색시작이므로 안건드려도 됨
search_path = '/html/body/echem-root/div/echem-substance-search-page/echem-substance-search-container/echem-substance-search/form/div/div[2]/div/button'
driver.find_element_by_xpath(search_path).click()

In [26]:
#%%
result_ = []

page_num_path = '/html/body/echem-root/div/echem-substance-search-page/echem-property-search-results-container/echem-pagination/div/div[2]'
page_num = int(driver.find_element_by_xpath(page_num_path).text.split(' ')[-1])

In [27]:
start = time.time()
for p in range(1, page_num + 1):
#for p in range(1, 4):
    row_num_path = '/html/body/echem-root/div/echem-substance-search-page/echem-property-search-results-container/echem-property-search-results/table/tbody/tr'
    row_num = len(driver.find_elements_by_xpath(row_num_path))
    
    row = tqdm(range(1, row_num + 1), file = sys.stdout)
    
    for i in row:
        src_dict = {}
        
        chem_path = '//*[@id="top"]/echem-substance-search-page/echem-property-search-results-container/echem-property-search-results/table/tbody/tr[%d]/td[3]/a'
        property_url = driver.find_element_by_xpath(chem_path % i).get_attribute('href')
        src_dict['link'] = property_url
        
        driver.execute_script('window.open('');')
        driver.switch_to.window(driver.window_handles[1])
        driver.get(property_url)
        time.sleep(1)
        
        try:
            accept_path = '/html/body/div[1]/div/div[2]/div[2]/button[1]'
            driver.find_element_by_xpath(accept_path).send_keys(Keys.ENTER)
            #driver.find_element_by_xpath(accept_path).click()
        except NoSuchElementException:
            pass
        
        try:
            src = driver.page_source
            # soup = BeautifulSoup(src, 'html.parser')
            src_dict['src'] = src
            
            result_.append(src_dict)
        
        except AttributeError:
            pass

        driver.close()
        driver.switch_to.window(driver.window_handles[0])
        
        row.set_postfix({'page': p})
    
    if p < page_num:
        next_page_path = '/html/body/echem-root/div/echem-substance-search-page/echem-property-search-results-container/echem-pagination/div/div[2]/a[3]'
        driver.find_element_by_xpath(next_page_path).click()
        time.sleep(1.5)
    
    p += 1


print(time.time() - start)


#%%
json.dump(result_, open('tg476_source.json', 'w'))

100%|███████████████████████████████████████████████████████████████████████████| 5/5 [00:13<00:00,  2.65s/it, page=172]
2872.956813097


In [28]:
##json전처리

In [2]:
import re
import json

import numpy as np
import pandas as pd

from tqdm import tqdm
from bs4 import BeautifulSoup


#%%
with open('tg476_source.json', 'r') as file:
    df = pd.DataFrame(json.load(file))

#%%
def remove_bracket(string):
    clean_string = re.sub('<.*?>', '', str(string))
    return clean_string


#%%
result_ = []

for i in tqdm(range(len(df))):
    try: 
        soup = BeautifulSoup(df.src[i], 'html.parser')
        chem_dict = {}
        
        # chemical name
        chem_name = soup.find('div', attrs = {'id': 'SubstanceName'}).find_next('h1').text
        chem_dict['Chemical'] = chem_name


        # casrn
        casrn_tmp = soup.find('div', attrs = {'class': 'container'}).find_next('strong').text
        casrn = re.sub('\n|\t', '', casrn_tmp).split( )[-1]
        chem_dict['CasRN'] = casrn
        
        
        # Metabolic activation
        try:
            admin_tmp = soup.find('h3', attrs = {'id': 'sResultsAndDiscussion'})
            
            admin_tmp = admin_tmp.find_next_siblings('dl')
            admin_idx = [remove_bracket(x.find_next('dt')) for x in admin_tmp].index('Metabolic activation:')  # 결과 확인
            
            route_tmp = admin_tmp[admin_idx]
            
            chem_dict['Metabolic activation'] = remove_bracket(route_tmp.find_next('dd'))
            
        except ValueError:
            chem_dict['Metabolic activation'] = np.nan
            
        # Genotoxicity
        try:
            admin_tmp = soup.find('h3', attrs = {'id': 'sResultsAndDiscussion'})
            
            admin_tmp = admin_tmp.find_next_siblings('dl')
            admin_idx = [remove_bracket(x.find_next('dt')) for x in admin_tmp].index('Genotoxicity:')  # 결과 확인
            
            route_tmp = admin_tmp[admin_idx]
            
            chem_dict['Genotoxicity'] = remove_bracket(route_tmp.find_next('dd'))
            
        except ValueError:
            chem_dict['Genotoxicity'] = np.nan
            
            
            
        # Body weight and weight changes
        try:
            admin_tmp = soup.find('h3', attrs = {'id': 'sApplicantSummaryAndConclusion'})
            
            admin_tmp = admin_tmp.find_next_siblings('dl')
            admin_idx = [remove_bracket(x.find_next('dt')) for x in admin_tmp].index('Conclusions:')  # 결과 확인
            
            route_tmp = admin_tmp[admin_idx]
            
            chem_dict['conclusion'] = remove_bracket(route_tmp.find_next('dd'))
            
        except ValueError:
            chem_dict['conclusion'] = np.nan
            
            
        # other results
        result_and_discussion = soup.find('h3', attrs={'id': 'sResultsAndDiscussion'})
        table_list = result_and_discussion.find_next_sibling('div').find_all('dl')
            
            
        for tab in table_list:
            chem_dict_ = chem_dict.copy()
            
            key = [re.sub(':', '', i.text).strip() for i in tab.find_all('dt')]
            value = [i.text.strip() for i in tab.find_all('dd')]
                                    
            if len(key) == len(value) and key[0] != '' and value[0] != 'Key result':
                result_dict = dict(zip(key, value))
                # result_dict = {key[i]: re.sub('<.*?>', '', cell.text).strip() for i, cell in enumerate(tab.find_all('dd'))}
            
            elif len(key) == len(value) and key[0] == '' and value[0] == 'Key result':
                result_dict = dict(zip(key[1:], value[1:]))
            
            elif len(key) != len(value) and key[0] == '' and value[0] == 'Key result':
                key = key[1:]
                value_ = value[1:len(key)] + ['. '.join(value[len(key):])]
                result_dict = dict(zip(key, value_))
            
            chem_dict_.update(result_dict)
            result_.append(chem_dict_)

    except AttributeError:
        pass
    


# %%
result = pd.DataFrame(result_)

100%|█████████████████████████████████████████████████████████████████████████████████| 860/860 [00:33<00:00, 25.85it/s]


In [3]:
result

Chemical       CasRN  \
0                      3-iodo-2-propynyl butylcarbamate  55406-53-6   
1                                Benzyl butyl phthalate     85-68-7   
2                 2-(4-tert-butylbenzyl)propionaldehyde     80-54-6   
3     3,3,4,4,5,5,6,6,7,7,8,8,8-tridecafluorooctyl m...   2144-53-8   
4     3,3,4,4,5,5,6,6,7,7,8,8,8-tridecafluorooctyl m...   2144-53-8   
...                                                 ...         ...   
1859             Cyclohexane-1,2-dicarboxylic anhydride     85-42-7   
1860                          Silicic acid, sodium salt   1344-09-8   
1861                          Silicic acid, sodium salt   1344-09-8   
1862                         2,2',2''-nitrilotriethanol    102-71-6   
1863                                 Butyl methacrylate     97-88-1   

      Clinical signs  Reproductive effects  \
0                NaN                   NaN   
1                NaN                   NaN   
2                NaN                   NaN   
3                NaN                   NaN   
4                NaN                   NaN   
...              ...                   ...   
1859             NaN                   NaN   
1860             NaN                   NaN   
1861             NaN                   NaN   
1862             NaN                   NaN   
1863             NaN                   NaN   

                                             conclusion  \
0     The test item was tested in the V79/HPRT-test ...   
1     Interpretation of results (migrated informatio...   
2                                                   NaN   
3     Interpretation of results (migrated informatio...   
4     Interpretation of results (migrated informatio...   
...                                                 ...   
1859  It is concluded that Hexahydrophthalic anhydri...   
1860                                                NaN   
1861                                                NaN   
1862                                                NaN   
1863  Interpretation of results (migrated informatio...   

                                       Species / strain Metabolic activation  \
0                Chinese hamster lung fibroblasts (V79)     with and without   
1                           Chinese hamster Ovary (CHO)     with and without   
2     other: Salmonella typhimurium strains TA98, TA...     with and without   
3     S. typhimurium TA 1535, TA 1537, TA 98 and TA 100     with and without   
4                                     E. coli WP2 uvr A     with and without   
...                                                 ...                  ...   
1859                          E. coli WP2 uvr A pKM 101     with and without   
1860             Chinese hamster lung fibroblasts (V79)     with and without   
1861             Chinese hamster lung fibroblasts (V79)     with and without   
1862                        Chinese hamster Ovary (CHO)     with and without   
1863  other: Salmonella typhimurium TA100, TA1535, T...     with and without   

     Genotoxicity        Cytotoxicity / choice of top concentrations  \
0        negative                                       cytotoxicity   
1        negative                                      not specified   
2        negative                                       cytotoxicity   
3        negative  no cytotoxicity, but tested up to precipitatin...   
4        negative  no cytotoxicity, but tested up to precipitatin...   
...           ...                                                ...   
1859     negative                                    no cytotoxicity   
1860     negative       other: 156.3 - 312.5 µg active ingredient/mL   
1861     negative       other: 156.3 - 312.5 µg active ingredient/mL   
1862     negative  no cytotoxicity nor precipitates, but tested u...   
1863     negative                                       cytotoxicity   

                                                Remarks  \
0     without S9 at 1 µg/mL and above; with S9 

In [31]:
result['Genotoxicity'].value_counts()

negative                                                                                                                                                                                                                         1708
positive                                                                                                                                                                                                                          135
ambiguous                                                                                                                                                                                                                          12
not determined                                                                                                                                                                                                                      4
other: no 5th strain tested due to guideline then in force                      

In [32]:
result['Metabolic activation'].value_counts()

with and without    1680
without               91
with                  90
not applicable         2
not specified          1
Name: Metabolic activation, dtype: int64

In [52]:

#%%
'''
    1. nan이 아닌 경우
    aerosol -> 보수적인 쪽으로. Dusts and Mists로 넣으면 됨 (aerosol include mists, smokes, fumes, and dusts)
    
    vapour + aerosol 같이 여러개가 혼합된 경우 보수적인 카테고리로 포함
    
    aerosol, vapour, gas, dust가 포함되는 애들 찾고
    aerosol이 포함되면 aerosol category
    aerosol 없이 vapour or vapor or vaporization 포함되면 vapour
    aerosol 없이 gas 포함되면 gas
    
    2. nan인경우 많은 쪽으로 포함
    3. inhalation만 있는 경우 많은 쪽으로 포함
    4. aerosol, vapour, gas, dust, mist가 포함되어있지 않으면 많은 쪽으로 포함
    
    1. effects observed가 하나라도 있으면 1
    1-1. no effects observed가 있으면 0
    
'''

def find_colon(string):
    return ':' in string


def check_nan(string):
    return string == string
        

# result['Route of administration'].unique()
# result['Route of administration'].value_counts()

# effect 항목의 결과 내용 중 포함이 될 경우
def effective(string):
    if check_nan(string):
        string = string.lower()
        if 'negative' in string:
            type_ = 0
        
        elif 'positive' in string:
            type_ = 1
            
        else:
            type_ = np.nan
    
    else:
        type_ = np.nan
        
    return type_

def conclusion(string):
    if check_nan(string):
        string = string.lower()
        #주요 단어를 적는 곳인듯
        if 'no effect' in string or 'no observed' in string or 'no evidence' in string or 'No reproductive toxicity' in string \
        or 'no reproductive toxicity' in string or 'absence' in string or 'not' in string:
            type_ = 0
        
        elif 'reduce' in string or 'decrease' in string:
            type_ = 10
        
        elif 'LOAEL' in string or 'LOEC' in string or 'LO' in string:
            type_ = 1
        else:
            type_ = np.nan
    
    else:
        type_ = np.nan
        
    return type_

#%%
df = result.copy()

# effect
df['target'] = df['Genotoxicity'].map(lambda x: effective(x))

# make for result
df = df[['CasRN','target']]

In [53]:
df.head()

CasRN Genotoxicity
0  55406-53-6     negative
1     85-68-7     negative
2     80-54-6     negative
3   2144-53-8     negative
4   2144-53-8     negative

In [54]:
# save df
df.to_excel('tg476_raw.xlsx', header = True, index = False)

# smiles 추가

In [55]:
import cirpy
import openpyxl
import warnings

import pandas as pd
import numpy as np

from tqdm import tqdm

pd.set_option('mode.chained_assignment', None)
warnings.filterwarnings("ignore")

In [59]:
data = pd.read_excel('tg476_raw.xlsx')
len(data['CasRN'].unique())

639

In [60]:
data.head()

CasRN Genotoxicity
0  55406-53-6     negative
1     85-68-7     negative
2     80-54-6     negative
3   2144-53-8     negative
4   2144-53-8     negative

In [63]:
data['target'].unique()
data['target'].isna().sum()

data = data[data['target'].notna()]

casrn_na_idx = data[data['CasRN'] == '-'].index

data = data.drop(casrn_na_idx).reset_index(drop = True)

target_na_idx = data[data['target'] == ''].index

data = data.drop(target_na_idx).reset_index(drop=True)

In [64]:
data = data.drop_duplicates()
len(data)

676

In [65]:
data1 = data[:300]
data2 = data[300:]

In [66]:
tqdm.pandas()
data1['SMILES'] = data1.CasRN.progress_apply(lambda x: cirpy.resolve(x, 'smiles'))

100%|█████████████████████████████████████████████████████████████████████████████████| 300/300 [06:31<00:00,  1.30s/it]


In [67]:
data1.to_excel('tg476_1.xlsx', header = True, index = False)

In [70]:
tqdm.pandas()
data2['SMILES'] = data2.CasRN.progress_apply(lambda x: cirpy.resolve(x, 'smiles'))

100%|█████████████████████████████████████████████████████████████████████████████████| 376/376 [07:51<00:00,  1.25s/it]


In [69]:
data2

CasRN Genotoxicity SMILES
300  26591-72-0     negative    NaN
301  91081-13-9     negative    NaN
302  68411-06-3     negative    NaN
303     50-89-5     negative    NaN
304  17865-07-5     negative    NaN
..          ...          ...    ...
671  62037-80-3     negative    NaN
672     85-42-7     negative    NaN
673   1344-09-8     negative    NaN
674    102-71-6     negative    NaN
675     97-88-1     negative    NaN

[376 rows x 3 columns]

In [72]:
data_concat=pd.concat([data1,data2])
data_concat

CasRN Genotoxicity  \
0    55406-53-6     negative   
1       85-68-7     negative   
2       80-54-6     negative   
3     2144-53-8     negative   
4      108-90-7     negative   
..          ...          ...   
671  62037-80-3     negative   
672     85-42-7     negative   
673   1344-09-8     negative   
674    102-71-6     negative   
675     97-88-1     negative   

                                                SMILES  
0                                     CCCCNC(=O)OCC#CI  
1                    CCCCOC(=O)c1ccccc1C(=O)OCc2ccccc2  
2                           CC(Cc1ccc(cc1)C(C)(C)C)C=O  
3    CC(=C)C(=O)OCCC(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)...  
4                                           Clc1ccccc1  
..                                                 ...  
671                                               None  
672                                O=C1OC(=O)C2CCCCC12  
673             O.O.O.O.O.[Na+].[Na+].[O-][Si]([O-])=O  
674                                       OCCN(CCO)CCO  
675                                   CCCCOC(=O)C(C)=C  

[676 rows x 3 columns]

In [ ]:
data_concat.to_excel('tg476_2.xlsx', header = True, index = False)

In [73]:
data_concat.SMILES.isna().sum()
data_concat = data_concat[data_concat['SMILES'].notna()].reset_index(drop = True)

In [74]:
data_concat

CasRN Genotoxicity  \
0    55406-53-6     negative   
1       85-68-7     negative   
2       80-54-6     negative   
3     2144-53-8     negative   
4      108-90-7     negative   
..          ...          ...   
434    100-41-4     negative   
435     85-42-7     negative   
436   1344-09-8     negative   
437    102-71-6     negative   
438     97-88-1     negative   

                                                SMILES  
0                                     CCCCNC(=O)OCC#CI  
1                    CCCCOC(=O)c1ccccc1C(=O)OCc2ccccc2  
2                           CC(Cc1ccc(cc1)C(C)(C)C)C=O  
3    CC(=C)C(=O)OCCC(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)...  
4                                           Clc1ccccc1  
..                                                 ...  
434                                         CCc1ccccc1  
435                                O=C1OC(=O)C2CCCCC12  
436             O.O.O.O.O.[Na+].[Na+].[O-][Si]([O-])=O  
437                                       OCCN(CCO)CCO  
438                                   CCCCOC(=O)C(C)=C  

[439 rows x 3 columns]

In [75]:
data_concat.to_excel('tg476.xlsx', header = True, index = False)